# 🚨 PIPELINE EXPLÍCITO: RASTREAMENTO COMPLETO DE FEATURES

## Estratégia de Correção:
1. **Pipeline Explícito**: Cada dataframe tem nome único e descritivo
2. **Auditoria de Penhascos**: Identificar exatamente onde features são perdidas
3. **Checkpoints com .info()**: Inspeção detalhada em cada etapa
4. **Preservação Rigorosa**: Garantir que nenhuma feature seja perdida

## Nomenclatura do Pipeline:
- `df_step01_raw`: Dados brutos carregados
- `df_step02_processed`: Dados processados básicos
- `df_step03_features`: Com features técnicas criadas
- `df_step04_selected`: Após seleção de features
- `df_step05_temporal`: Após correção temporal
- `df_step06_candlestick`: Com padrões de candlestick
- `df_step07_final`: Dataset final para modelagem

In [1]:
# FUNÇÃO DE AUDITORIA RIGOROSA
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

def audit_pipeline_step(df, step_name, expected_min_cols=None, previous_df=None):
    """
    Auditoria rigorosa de cada etapa do pipeline
    Identifica exatamente onde features são perdidas
    """
    print(f"\n{'='*60}")
    print(f"🔍 AUDITORIA: {step_name.upper()}")
    print(f"{'='*60}")
    
    # Informações básicas
    print(f"📊 INFORMAÇÕES BÁSICAS:")
    print(f"   Shape: {df.shape}")
    print(f"   Memória: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")
    
    # Usar .info() para inspeção detalhada
    print(f"\n📋 DETALHES DAS COLUNAS (.info()):")
    df.info()
    
    # Categorização de features
    ohlc_cols = [col for col in df.columns if col in ['Abertura', 'Máxima', 'Mínima', 'Último', 'Volume']]
    temporal_cols = [col for col in df.columns if any(x in col.lower() for x in ['data', 'day', 'month', 'quarter', 'year'])]
    technical_cols = [col for col in df.columns if any(x in col.lower() for x in ['ma_', 'bb_', 'rsi', 'atr', 'volatility', 'macd'])]
    shifted_cols = [col for col in df.columns if col.endswith('_shifted')]
    candlestick_cols = [col for col in df.columns if any(x in col.lower() for x in ['doji', 'hammer', 'engulf', 'marubozu', 'shooting', '_prev'])]
    target_cols = [col for col in df.columns if 'target' in col.lower()]
    other_cols = [col for col in df.columns if col not in ohlc_cols + temporal_cols + technical_cols + shifted_cols + candlestick_cols + target_cols]
    
    print(f"\n📂 CATEGORIZAÇÃO DE FEATURES:")
    print(f"   🏢 OHLC/Volume ({len(ohlc_cols)}): {ohlc_cols}")
    print(f"   📅 Temporais ({len(temporal_cols)}): {temporal_cols}")
    print(f"   📈 Técnicas ({len(technical_cols)}): {technical_cols[:5]}{'...' if len(technical_cols) > 5 else ''}")
    print(f"   ⏰ Shifted ({len(shifted_cols)}): {shifted_cols[:5]}{'...' if len(shifted_cols) > 5 else ''}")
    print(f"   🕯️ Candlestick ({len(candlestick_cols)}): {candlestick_cols[:5]}{'...' if len(candlestick_cols) > 5 else ''}")
    print(f"   🎯 Target ({len(target_cols)}): {target_cols}")
    print(f"   ❓ Outras ({len(other_cols)}): {other_cols[:5]}{'...' if len(other_cols) > 5 else ''}")
    
    # Verificar se há perda de features
    if previous_df is not None:
        print(f"\n🚨 ANÁLISE DE MUDANÇAS:")
        prev_cols = set(previous_df.columns)
        curr_cols = set(df.columns)
        
        lost_features = prev_cols - curr_cols
        gained_features = curr_cols - prev_cols
        
        print(f"   📉 Features PERDIDAS ({len(lost_features)}):")
        if lost_features:
            for feature in sorted(lost_features):
                print(f"      ❌ {feature}")
        else:
            print(f"      ✅ Nenhuma feature perdida")
        
        print(f"   📈 Features GANHAS ({len(gained_features)}):")
        if gained_features:
            for feature in sorted(gained_features):
                print(f"      ✅ {feature}")
        else:
            print(f"      ➖ Nenhuma feature ganha")
        
        net_change = len(curr_cols) - len(prev_cols)
        print(f"   📊 Mudança líquida: {net_change:+d} features")
        
        # ALERTA para perdas significativas
        if len(lost_features) > 5:
            print(f"\n🚨 ALERTA: PERDA SIGNIFICATIVA DE FEATURES!")
            print(f"   {len(lost_features)} features foram perdidas nesta etapa")
            print(f"   Isso pode indicar um 'PENHASCO' no pipeline")
    
    # Verificar expectativas
    if expected_min_cols and len(df.columns) < expected_min_cols:
        print(f"\n⚠️ AVISO: Menos colunas que esperado")
        print(f"   Esperado: >= {expected_min_cols}")
        print(f"   Atual: {len(df.columns)}")
    
    # Verificar qualidade dos dados
    print(f"\n🔍 QUALIDADE DOS DADOS:")
    null_counts = df.isnull().sum()
    cols_with_nulls = null_counts[null_counts > 0]
    print(f"   Colunas com valores ausentes: {len(cols_with_nulls)}")
    if len(cols_with_nulls) > 0:
        print(f"   Top 5 com mais NaNs:")
        for col, count in cols_with_nulls.head().items():
            print(f"      {col}: {count} ({count/len(df):.1%})")
    
    print(f"\n✅ Auditoria de {step_name} concluída")
    print(f"{'='*60}")
    
    return {
        'step_name': step_name,
        'shape': df.shape,
        'columns': list(df.columns),
        'ohlc_count': len(ohlc_cols),
        'technical_count': len(technical_cols),
        'shifted_count': len(shifted_cols),
        'candlestick_count': len(candlestick_cols)
    }

print("✅ Função de auditoria rigorosa criada")
print("📋 Pronta para rastrear cada etapa do pipeline")

✅ Função de auditoria rigorosa criada
📋 Pronta para rastrear cada etapa do pipeline


## STEP 01: CARREGAMENTO DOS DADOS BRUTOS

In [2]:
# STEP 01: CARREGAMENTO DOS DADOS BRUTOS
print("🚀 INICIANDO PIPELINE EXPLÍCITO")
print("📂 STEP 01: Carregando dados brutos...")

# Carregar dados brutos
df_step01_raw = pd.read_csv('Dados Históricos - Ibovespa.csv', encoding='utf-8')

# Auditoria do carregamento
audit_step01 = audit_pipeline_step(df_step01_raw, "STEP 01 - Dados Brutos")

print(f"\n📋 RESUMO STEP 01:")
print(f"   ✅ Dados carregados: {df_step01_raw.shape}")
print(f"   📊 Colunas originais: {list(df_step01_raw.columns)}")

🚀 INICIANDO PIPELINE EXPLÍCITO
📂 STEP 01: Carregando dados brutos...

🔍 AUDITORIA: STEP 01 - DADOS BRUTOS
📊 INFORMAÇÕES BÁSICAS:
   Shape: (3592, 7)
   Memória: 0.68 MB

📋 DETALHES DAS COLUNAS (.info()):
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3592 entries, 0 to 3591
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Data      3592 non-null   object 
 1   Último    3592 non-null   float64
 2   Abertura  3592 non-null   float64
 3   Máxima    3592 non-null   float64
 4   Mínima    3592 non-null   float64
 5   Vol.      3591 non-null   object 
 6   Var%      3592 non-null   object 
dtypes: float64(4), object(3)
memory usage: 196.6+ KB

📂 CATEGORIZAÇÃO DE FEATURES:
   🏢 OHLC/Volume (4): ['Último', 'Abertura', 'Máxima', 'Mínima']
   📅 Temporais (1): ['Data']
   📈 Técnicas (0): []
   ⏰ Shifted (0): []
   🕯️ Candlestick (0): []
   🎯 Target (0): []
   ❓ Outras (2): ['Vol.', 'Var%']

🔍 QUALIDADE DOS DADOS:
   Colunas com

## STEP 02: PROCESSAMENTO BÁSICO

In [3]:
# STEP 02: PROCESSAMENTO BÁSICO
print("\n📂 STEP 02: Processamento básico...")

# Criar cópia para processamento
df_step02_processed = df_step01_raw.copy()

# Processamento de data
df_step02_processed['Data'] = pd.to_datetime(df_step02_processed['Data'], format='%d.%m.%Y')
df_step02_processed = df_step02_processed.sort_values('Data').reset_index(drop=True)

# Tratar valores ausentes
df_step02_processed['Vol.'] = df_step02_processed['Vol.'].fillna(method='ffill')

# Converter Volume para numérico
def converter_volume(vol_str):
    if pd.isna(vol_str): return np.nan
    vol_str = str(vol_str).replace(',', '.')
    if 'B' in vol_str: return float(vol_str.replace('B', '')) * 1e9
    elif 'M' in vol_str: return float(vol_str.replace('M', '')) * 1e6
    elif 'K' in vol_str: return float(vol_str.replace('K', '')) * 1e3
    return float(vol_str)

df_step02_processed['Volume'] = df_step02_processed['Vol.'].apply(converter_volume)
df_step02_processed['Variacao'] = df_step02_processed['Var%'].str.replace('%', '').str.replace(',', '.').astype(float) / 100

# Criar target
df_step02_processed['Target'] = (df_step02_processed['Variacao'].shift(-1) > 0).astype(int)
df_step02_processed = df_step02_processed[:-1].copy()  # Remove última linha

# Auditoria do processamento
audit_step02 = audit_pipeline_step(
    df_step02_processed, 
    "STEP 02 - Processamento Básico", 
    expected_min_cols=len(df_step01_raw.columns),
    previous_df=df_step01_raw
)

print(f"\n📋 RESUMO STEP 02:")
print(f"   ✅ Target criado corretamente")
print(f"   📊 Shape após processamento: {df_step02_processed.shape}")
print(f"   🎯 Distribuição do target:")
print(df_step02_processed['Target'].value_counts(normalize=True))


📂 STEP 02: Processamento básico...

🔍 AUDITORIA: STEP 02 - PROCESSAMENTO BÁSICO
📊 INFORMAÇÕES BÁSICAS:
   Shape: (3591, 10)
   Memória: 0.59 MB

📋 DETALHES DAS COLUNAS (.info()):
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3591 entries, 0 to 3590
Data columns (total 10 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   Data      3591 non-null   datetime64[ns]
 1   Último    3591 non-null   float64       
 2   Abertura  3591 non-null   float64       
 3   Máxima    3591 non-null   float64       
 4   Mínima    3591 non-null   float64       
 5   Vol.      3591 non-null   object        
 6   Var%      3591 non-null   object        
 7   Volume    3591 non-null   float64       
 8   Variacao  3591 non-null   float64       
 9   Target    3591 non-null   int64         
dtypes: datetime64[ns](1), float64(6), int64(1), object(2)
memory usage: 280.7+ KB

📂 CATEGORIZAÇÃO DE FEATURES:
   🏢 OHLC/Volume (5): ['Último', 'Abertura', 'Má

## STEP 03: CRIAÇÃO DE FEATURES TÉCNICAS

In [4]:
# STEP 03: CRIAÇÃO DE FEATURES TÉCNICAS
print("\n📂 STEP 03: Criando features técnicas...")

# Criar cópia para features
df_step03_features = df_step02_processed.copy()

print("📈 Criando indicadores técnicos...")

# Médias móveis
for periodo in [5, 10, 20, 50]:
    df_step03_features[f'MA_{periodo}'] = df_step03_features['Último'].rolling(window=periodo).mean()

# Bandas de Bollinger
df_step03_features['BB_Middle'] = df_step03_features['Último'].rolling(window=20).mean()
bb_std = df_step03_features['Último'].rolling(window=20).std()
df_step03_features['BB_Upper'] = df_step03_features['BB_Middle'] + (bb_std * 2)
df_step03_features['BB_Lower'] = df_step03_features['BB_Middle'] - (bb_std * 2)
df_step03_features['BB_Width'] = df_step03_features['BB_Upper'] - df_step03_features['BB_Lower']
df_step03_features['BB_Position'] = (df_step03_features['Último'] - df_step03_features['BB_Lower']) / df_step03_features['BB_Width']

# RSI
def calculate_rsi(prices, window=14):
    delta = prices.diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=window).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=window).mean()
    rs = gain / loss
    return 100 - (100 / (1 + rs))

df_step03_features['RSI'] = calculate_rsi(df_step03_features['Último'])

# MACD
ema_12 = df_step03_features['Último'].ewm(span=12).mean()
ema_26 = df_step03_features['Último'].ewm(span=26).mean()
df_step03_features['MACD'] = ema_12 - ema_26
df_step03_features['Signal_Line'] = df_step03_features['MACD'].ewm(span=9).mean()

# ATR (Average True Range)
df_step03_features['high_low'] = df_step03_features['Máxima'] - df_step03_features['Mínima']
df_step03_features['high_close_prev'] = abs(df_step03_features['Máxima'] - df_step03_features['Último'].shift(1))
df_step03_features['low_close_prev'] = abs(df_step03_features['Mínima'] - df_step03_features['Último'].shift(1))
df_step03_features['true_range'] = df_step03_features[['high_low', 'high_close_prev', 'low_close_prev']].max(axis=1)

for periodo in [5, 10, 20]:
    df_step03_features[f'atr_{periodo}'] = df_step03_features['true_range'].rolling(window=periodo).mean()

# Volatilidade
df_step03_features['returns'] = df_step03_features['Último'].pct_change()
for periodo in [5, 10, 20]:
    df_step03_features[f'volatility_{periodo}'] = df_step03_features['returns'].rolling(window=periodo).std()

# Features de preço
df_step03_features['Price_Range'] = df_step03_features['Máxima'] - df_step03_features['Mínima']
df_step03_features['Price_Position'] = (df_step03_features['Último'] - df_step03_features['Mínima']) / df_step03_features['Price_Range']
df_step03_features['Gap'] = df_step03_features['Abertura'] - df_step03_features['Último'].shift(1)
df_step03_features['hl_close_ratio'] = (df_step03_features['Máxima'] - df_step03_features['Mínima']) / df_step03_features['Último']

# Features temporais
df_step03_features['day_of_week'] = df_step03_features['Data'].dt.dayofweek
df_step03_features['month'] = df_step03_features['Data'].dt.month
df_step03_features['quarter'] = df_step03_features['Data'].dt.quarter
df_step03_features['year'] = df_step03_features['Data'].dt.year
df_step03_features['is_month_start'] = (df_step03_features['Data'].dt.day <= 5).astype(int)
df_step03_features['is_month_end'] = (df_step03_features['Data'].dt.day >= 25).astype(int)
df_step03_features['is_quarter_end'] = df_step03_features['Data'].dt.is_quarter_end.astype(int)

# Auditoria da criação de features
audit_step03 = audit_pipeline_step(
    df_step03_features, 
    "STEP 03 - Features Técnicas", 
    expected_min_cols=30,  # Esperamos pelo menos 30 features
    previous_df=df_step02_processed
)

print(f"\n📋 RESUMO STEP 03:")
print(f"   ✅ Features técnicas criadas: {df_step03_features.shape[1] - df_step02_processed.shape[1]}")
print(f"   📊 Total de features: {df_step03_features.shape[1]}")
print(f"   🎯 Shape final: {df_step03_features.shape}")


📂 STEP 03: Criando features técnicas...
📈 Criando indicadores técnicos...

🔍 AUDITORIA: STEP 03 - FEATURES TÉCNICAS
📊 INFORMAÇÕES BÁSICAS:
   Shape: (3591, 44)
   Memória: 1.47 MB

📋 DETALHES DAS COLUNAS (.info()):
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3591 entries, 0 to 3590
Data columns (total 44 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   Data             3591 non-null   datetime64[ns]
 1   Último           3591 non-null   float64       
 2   Abertura         3591 non-null   float64       
 3   Máxima           3591 non-null   float64       
 4   Mínima           3591 non-null   float64       
 5   Vol.             3591 non-null   object        
 6   Var%             3591 non-null   object        
 7   Volume           3591 non-null   float64       
 8   Variacao         3591 non-null   float64       
 9   Target           3591 non-null   int64         
 10  MA_5             3587 non-null   flo

## STEP 04: SELEÇÃO DE FEATURES (PONTO CRÍTICO)

In [5]:
# STEP 04: SELEÇÃO DE FEATURES - PONTO CRÍTICO!
print("\n📂 STEP 04: Seleção de features (PONTO CRÍTICO)...")
print("🚨 ATENÇÃO: Esta é uma área comum para 'PENHASCOS' de features!")

# ANTES da seleção - checkpoint crítico
print(f"\n🔍 CHECKPOINT ANTES DA SELEÇÃO:")
print(f"   Features disponíveis: {df_step03_features.shape[1]}")
print(f"   Primeiras 10: {list(df_step03_features.columns)[:10]}")
print(f"   Últimas 10: {list(df_step03_features.columns)[-10:]}")

# Criar cópia para seleção
df_step04_selected = df_step03_features.copy()

# ESTRATÉGIA CONSERVADORA: Manter TODAS as features criadas
# Apenas remover colunas auxiliares e duplicadas
columns_to_remove = [
    'Vol.',  # Versão string do volume
    'Var%',  # Versão string da variação
    'high_low',  # Auxiliar para ATR
    'BB_Middle'  # Duplicata da MA_20
]

# Remover apenas colunas auxiliares
columns_to_remove_existing = [col for col in columns_to_remove if col in df_step04_selected.columns]
if columns_to_remove_existing:
    print(f"\n🗑️ Removendo apenas colunas auxiliares: {columns_to_remove_existing}")
    df_step04_selected = df_step04_selected.drop(columns=columns_to_remove_existing)
else:
    print(f"\n✅ Nenhuma coluna auxiliar para remover")

# CHECKPOINT CRÍTICO: Verificar se não perdemos features importantes
features_importantes = [
    'Abertura', 'Máxima', 'Mínima', 'Último', 'Volume',
    'MA_5', 'MA_10', 'MA_20', 'MA_50',
    'BB_Upper', 'BB_Lower', 'BB_Width', 'BB_Position',
    'RSI', 'MACD', 'Signal_Line',
    'atr_5', 'atr_10', 'atr_20',
    'volatility_5', 'volatility_10', 'volatility_20',
    'Price_Range', 'Price_Position', 'Gap', 'hl_close_ratio',
    'day_of_week', 'month', 'quarter', 'year',
    'is_month_start', 'is_month_end', 'is_quarter_end',
    'Target'
]

features_perdidas = [f for f in features_importantes if f not in df_step04_selected.columns]
if features_perdidas:
    print(f"\n🚨 ALERTA: Features importantes perdidas: {features_perdidas}")
else:
    print(f"\n✅ Todas as features importantes preservadas")

# Auditoria da seleção
audit_step04 = audit_pipeline_step(
    df_step04_selected, 
    "STEP 04 - Seleção de Features", 
    expected_min_cols=len(features_importantes),
    previous_df=df_step03_features
)

print(f"\n📋 RESUMO STEP 04:")
print(f"   ✅ Features selecionadas: {df_step04_selected.shape[1]}")
print(f"   📊 Features removidas: {df_step03_features.shape[1] - df_step04_selected.shape[1]}")
print(f"   🎯 Shape final: {df_step04_selected.shape}")


📂 STEP 04: Seleção de features (PONTO CRÍTICO)...
🚨 ATENÇÃO: Esta é uma área comum para 'PENHASCOS' de features!

🔍 CHECKPOINT ANTES DA SELEÇÃO:
   Features disponíveis: 44
   Primeiras 10: ['Data', 'Último', 'Abertura', 'Máxima', 'Mínima', 'Vol.', 'Var%', 'Volume', 'Variacao', 'Target']
   Últimas 10: ['Price_Position', 'Gap', 'hl_close_ratio', 'day_of_week', 'month', 'quarter', 'year', 'is_month_start', 'is_month_end', 'is_quarter_end']

🗑️ Removendo apenas colunas auxiliares: ['Vol.', 'Var%', 'high_low', 'BB_Middle']

✅ Todas as features importantes preservadas

🔍 AUDITORIA: STEP 04 - SELEÇÃO DE FEATURES
📊 INFORMAÇÕES BÁSICAS:
   Shape: (3591, 40)
   Memória: 1.04 MB

📋 DETALHES DAS COLUNAS (.info()):
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3591 entries, 0 to 3590
Data columns (total 40 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   Data             3591 non-null   datetime64[ns]
 1   Último        

## STEP 05: CORREÇÃO TEMPORAL (PENHASCO CRÍTICO!)

In [6]:
# STEP 05: CORREÇÃO TEMPORAL - PENHASCO CRÍTICO!
print("\n📂 STEP 05: Correção temporal (PENHASCO CRÍTICO!)...")
print("🚨 ATENÇÃO: Esta é a área mais provável para perda massiva de features!")

# CHECKPOINT ANTES DA CORREÇÃO TEMPORAL
print(f"\n🔍 CHECKPOINT ANTES DA CORREÇÃO TEMPORAL:")
print(f"   Features disponíveis: {df_step04_selected.shape[1]}")
print(f"   Shape: {df_step04_selected.shape}")

# Criar cópia para correção temporal
df_step05_temporal = df_step04_selected.copy()

# ESTRATÉGIA CONSERVADORA: Aplicar shift apenas onde necessário
# Identificar features que precisam de shift (dados do presente/futuro)
features_to_shift = [
    'Abertura', 'Máxima', 'Mínima', 'Último', 'Volume',
    'MA_5', 'MA_10', 'MA_20', 'MA_50',
    'BB_Upper', 'BB_Lower', 'BB_Width', 'BB_Position',
    'RSI', 'MACD', 'Signal_Line',
    'atr_5', 'atr_10', 'atr_20',
    'volatility_5', 'volatility_10', 'volatility_20',
    'Price_Range', 'Price_Position', 'Gap', 'hl_close_ratio',
    'true_range', 'high_close_prev', 'low_close_prev',
    'returns', 'Variacao'
]

# Features que NÃO precisam de shift (dados temporais do passado)
features_no_shift = [
    'Data', 'day_of_week', 'month', 'quarter', 'year',
    'is_month_start', 'is_month_end', 'is_quarter_end',
    'Target'
]

# Verificar quais features estão disponíveis
available_to_shift = [f for f in features_to_shift if f in df_step05_temporal.columns]
available_no_shift = [f for f in features_no_shift if f in df_step05_temporal.columns]

print(f"\n📊 ANÁLISE DE FEATURES PARA SHIFT:")
print(f"   Features para shift: {len(available_to_shift)} de {len(features_to_shift)}")
print(f"   Features sem shift: {len(available_no_shift)} de {len(features_no_shift)}")

missing_to_shift = [f for f in features_to_shift if f not in df_step05_temporal.columns]
if missing_to_shift:
    print(f"   ⚠️ Features ausentes para shift: {missing_to_shift}")

# APLICAR SHIFT DE FORMA CONSERVADORA
print(f"\n⏰ Aplicando shift temporal...")

# Criar dataset final com features shifted
df_temporal_final = pd.DataFrame()

# 1. Manter features temporais sem shift
for feature in available_no_shift:
    df_temporal_final[feature] = df_step05_temporal[feature]

# 2. Aplicar shift nas features de preço/indicadores
for feature in available_to_shift:
    df_temporal_final[f'{feature}_shifted'] = df_step05_temporal[feature].shift(1)

# 3. Remover primeira linha (NaN devido ao shift)
df_temporal_final = df_temporal_final.iloc[1:].reset_index(drop=True)

# CHECKPOINT CRÍTICO APÓS CORREÇÃO TEMPORAL
print(f"\n🔍 CHECKPOINT APÓS CORREÇÃO TEMPORAL:")
print(f"   Features antes: {df_step05_temporal.shape[1]}")
print(f"   Features depois: {df_temporal_final.shape[1]}")
print(f"   Mudança: {df_temporal_final.shape[1] - df_step05_temporal.shape[1]:+d}")

# Verificar se temos as features shifted esperadas
expected_shifted = [f'{f}_shifted' for f in available_to_shift]
actual_shifted = [col for col in df_temporal_final.columns if col.endswith('_shifted')]

print(f"\n📊 VERIFICAÇÃO DE FEATURES SHIFTED:")
print(f"   Esperadas: {len(expected_shifted)}")
print(f"   Criadas: {len(actual_shifted)}")

missing_shifted = [f for f in expected_shifted if f not in df_temporal_final.columns]
if missing_shifted:
    print(f"   ❌ Features shifted ausentes: {missing_shifted[:5]}{'...' if len(missing_shifted) > 5 else ''}")
else:
    print(f"   ✅ Todas as features shifted criadas corretamente")

# Atualizar variável para próxima etapa
df_step05_temporal = df_temporal_final.copy()

# Auditoria da correção temporal
audit_step05 = audit_pipeline_step(
    df_step05_temporal, 
    "STEP 05 - Correção Temporal", 
    expected_min_cols=len(available_no_shift) + len(available_to_shift),
    previous_df=df_step04_selected
)

print(f"\n📋 RESUMO STEP 05:")
print(f"   ✅ Correção temporal aplicada")
print(f"   📊 Features shifted criadas: {len(actual_shifted)}")
print(f"   🎯 Shape final: {df_step05_temporal.shape}")


📂 STEP 05: Correção temporal (PENHASCO CRÍTICO!)...
🚨 ATENÇÃO: Esta é a área mais provável para perda massiva de features!

🔍 CHECKPOINT ANTES DA CORREÇÃO TEMPORAL:
   Features disponíveis: 40
   Shape: (3591, 40)

📊 ANÁLISE DE FEATURES PARA SHIFT:
   Features para shift: 31 de 31
   Features sem shift: 9 de 9

⏰ Aplicando shift temporal...

🔍 CHECKPOINT APÓS CORREÇÃO TEMPORAL:
   Features antes: 40
   Features depois: 40
   Mudança: +0

📊 VERIFICAÇÃO DE FEATURES SHIFTED:
   Esperadas: 31
   Criadas: 31
   ✅ Todas as features shifted criadas corretamente

🔍 AUDITORIA: STEP 05 - CORREÇÃO TEMPORAL
📊 INFORMAÇÕES BÁSICAS:
   Shape: (3590, 40)
   Memória: 1.04 MB

📋 DETALHES DAS COLUNAS (.info()):
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3590 entries, 0 to 3589
Data columns (total 40 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   Data                     3590 non-null   datetime64[ns]
 1   da

## STEP 06: PADRÕES DE CANDLESTICK

In [7]:
# STEP 06: PADRÕES DE CANDLESTICK
print("\n📂 STEP 06: Criando padrões de candlestick...")

# Criar cópia para candlestick
df_step06_candlestick = df_step05_temporal.copy()

# Verificar se temos as colunas OHLC shifted necessárias
ohlc_shifted_cols = ['Abertura_shifted', 'Máxima_shifted', 'Mínima_shifted', 'Último_shifted']
available_ohlc_shifted = [col for col in ohlc_shifted_cols if col in df_step06_candlestick.columns]

print(f"\n🕯️ VERIFICAÇÃO DE COLUNAS OHLC SHIFTED:")
print(f"   Necessárias: {ohlc_shifted_cols}")
print(f"   Disponíveis: {available_ohlc_shifted}")

if len(available_ohlc_shifted) == 4:
    print(f"   ✅ Todas as colunas OHLC shifted disponíveis")
    
    # Renomear temporariamente para facilitar cálculos
    ohlc_temp = df_step06_candlestick[available_ohlc_shifted].copy()
    ohlc_temp.columns = ['Open', 'High', 'Low', 'Close']
    
    # Funções de padrões de candlestick
    def detect_doji(df, threshold=0.1):
        body_size = abs(df['Close'] - df['Open'])
        total_range = df['High'] - df['Low']
        return (body_size / total_range < threshold).astype(int)
    
    def detect_hammer(df):
        body_size = abs(df['Close'] - df['Open'])
        lower_shadow = df[['Open', 'Close']].min(axis=1) - df['Low']
        upper_shadow = df['High'] - df[['Open', 'Close']].max(axis=1)
        return ((lower_shadow > 2 * body_size) & (upper_shadow < body_size)).astype(int)
    
    def detect_shooting_star(df):
        body_size = abs(df['Close'] - df['Open'])
        lower_shadow = df[['Open', 'Close']].min(axis=1) - df['Low']
        upper_shadow = df['High'] - df[['Open', 'Close']].max(axis=1)
        return ((upper_shadow > 2 * body_size) & (lower_shadow < body_size)).astype(int)
    
    def detect_engulfing_bullish(df):
        current_bullish = df['Close'] > df['Open']
        prev_bearish = (df['Close'].shift(1) < df['Open'].shift(1))
        current_open_below_prev_close = df['Open'] < df['Close'].shift(1)
        current_close_above_prev_open = df['Close'] > df['Open'].shift(1)
        return (current_bullish & prev_bearish & current_open_below_prev_close & current_close_above_prev_open).astype(int)
    
    # Aplicar padrões
    print(f"\n📊 Criando padrões de candlestick...")
    
    df_step06_candlestick['doji_prev'] = detect_doji(ohlc_temp)
    df_step06_candlestick['hammer_prev'] = detect_hammer(ohlc_temp)
    df_step06_candlestick['shooting_star_prev'] = detect_shooting_star(ohlc_temp)
    df_step06_candlestick['engulfing_bullish_prev'] = detect_engulfing_bullish(ohlc_temp)
    
    # Padrões adicionais
    df_step06_candlestick['bullish_candle_prev'] = (ohlc_temp['Close'] > ohlc_temp['Open']).astype(int)
    df_step06_candlestick['bearish_candle_prev'] = (ohlc_temp['Close'] < ohlc_temp['Open']).astype(int)
    
    # Métricas de candlestick
    df_step06_candlestick['body_size_prev'] = abs(ohlc_temp['Close'] - ohlc_temp['Open'])
    df_step06_candlestick['upper_shadow_prev'] = ohlc_temp['High'] - ohlc_temp[['Open', 'Close']].max(axis=1)
    df_step06_candlestick['lower_shadow_prev'] = ohlc_temp[['Open', 'Close']].min(axis=1) - ohlc_temp['Low']
    
    candlestick_features_created = 9
    print(f"   ✅ {candlestick_features_created} features de candlestick criadas")
    
else:
    print(f"   ❌ Colunas OHLC shifted insuficientes")
    print(f"   Ausentes: {[col for col in ohlc_shifted_cols if col not in df_step06_candlestick.columns]}")
    candlestick_features_created = 0

# Auditoria dos padrões de candlestick
audit_step06 = audit_pipeline_step(
    df_step06_candlestick, 
    "STEP 06 - Padrões de Candlestick", 
    expected_min_cols=df_step05_temporal.shape[1] + candlestick_features_created,
    previous_df=df_step05_temporal
)

print(f"\n📋 RESUMO STEP 06:")
print(f"   ✅ Features de candlestick criadas: {candlestick_features_created}")
print(f"   📊 Total de features: {df_step06_candlestick.shape[1]}")
print(f"   🎯 Shape final: {df_step06_candlestick.shape}")


📂 STEP 06: Criando padrões de candlestick...

🕯️ VERIFICAÇÃO DE COLUNAS OHLC SHIFTED:
   Necessárias: ['Abertura_shifted', 'Máxima_shifted', 'Mínima_shifted', 'Último_shifted']
   Disponíveis: ['Abertura_shifted', 'Máxima_shifted', 'Mínima_shifted', 'Último_shifted']
   ✅ Todas as colunas OHLC shifted disponíveis

📊 Criando padrões de candlestick...
   ✅ 9 features de candlestick criadas

🔍 AUDITORIA: STEP 06 - PADRÕES DE CANDLESTICK
📊 INFORMAÇÕES BÁSICAS:
   Shape: (3590, 49)
   Memória: 1.29 MB

📋 DETALHES DAS COLUNAS (.info()):
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3590 entries, 0 to 3589
Data columns (total 49 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   Data                     3590 non-null   datetime64[ns]
 1   day_of_week              3590 non-null   int32         
 2   month                    3590 non-null   int32         
 3   quarter                  3590 non-null   int3

## STEP 07: DATASET FINAL E AUDITORIA COMPLETA

In [8]:
# STEP 07: DATASET FINAL E AUDITORIA COMPLETA
print("\n📂 STEP 07: Preparando dataset final...")

# Criar dataset final
df_step07_final = df_step06_candlestick.copy()

# Limpeza final - remover linhas com muitos NaN
print(f"\n🧹 Limpeza final de dados...")
print(f"   Shape antes da limpeza: {df_step07_final.shape}")

# Contar NaN por linha
nan_counts = df_step07_final.isnull().sum(axis=1)
threshold = df_step07_final.shape[1] * 0.3  # Remover linhas com mais de 30% de NaN

df_step07_final = df_step07_final[nan_counts <= threshold]
print(f"   Shape após limpeza: {df_step07_final.shape}")
print(f"   Linhas removidas: {df_step06_candlestick.shape[0] - df_step07_final.shape[0]}")

# Auditoria final
audit_step07 = audit_pipeline_step(
    df_step07_final, 
    "STEP 07 - Dataset Final", 
    expected_min_cols=50,  # Esperamos pelo menos 50 features
    previous_df=df_step06_candlestick
)

print(f"\n📋 RESUMO STEP 07:")
print(f"   ✅ Dataset final preparado")
print(f"   📊 Features finais: {df_step07_final.shape[1]}")
print(f"   🎯 Shape final: {df_step07_final.shape}")
print(f"   💾 Pronto para modelagem!")


📂 STEP 07: Preparando dataset final...

🧹 Limpeza final de dados...
   Shape antes da limpeza: (3590, 49)
   Shape após limpeza: (3586, 49)
   Linhas removidas: 4

🔍 AUDITORIA: STEP 07 - DATASET FINAL
📊 INFORMAÇÕES BÁSICAS:
   Shape: (3586, 49)
   Memória: 1.31 MB

📋 DETALHES DAS COLUNAS (.info()):
<class 'pandas.core.frame.DataFrame'>
Index: 3586 entries, 4 to 3589
Data columns (total 49 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   Data                     3586 non-null   datetime64[ns]
 1   day_of_week              3586 non-null   int32         
 2   month                    3586 non-null   int32         
 3   quarter                  3586 non-null   int32         
 4   year                     3586 non-null   int32         
 5   is_month_start           3586 non-null   int64         
 6   is_month_end             3586 non-null   int64         
 7   is_quarter_end           3586 non-null   in

## AUDITORIA COMPLETA DO PIPELINE

In [9]:
# AUDITORIA COMPLETA DO PIPELINE
print("\n" + "="*80)
print("🔍 AUDITORIA COMPLETA DO PIPELINE EXPLÍCITO")
print("="*80)

# Coletar todos os resultados de auditoria
pipeline_steps = [
    audit_step01, audit_step02, audit_step03, 
    audit_step04, audit_step05, audit_step06, audit_step07
]

# Tabela resumo
print(f"\n📊 RESUMO COMPLETO DO PIPELINE:")
print(f"{'Step':<25} {'Shape':<15} {'Features':<10} {'Técnicas':<10} {'Shifted':<10} {'Candlestick':<12}")
print("-" * 85)

for step in pipeline_steps:
    step_name = step['step_name'][:23] + '...' if len(step['step_name']) > 25 else step['step_name']
    shape_str = f"{step['shape'][0]}x{step['shape'][1]}"
    print(f"{step_name:<25} {shape_str:<15} {step['shape'][1]:<10} {step['technical_count']:<10} {step['shifted_count']:<10} {step['candlestick_count']:<12}")

# Análise de evolução
print(f"\n📈 EVOLUÇÃO DAS FEATURES:")
for i in range(1, len(pipeline_steps)):
    prev_step = pipeline_steps[i-1]
    curr_step = pipeline_steps[i]
    
    change = curr_step['shape'][1] - prev_step['shape'][1]
    change_str = f"{change:+d}" if change != 0 else "0"
    
    print(f"   {prev_step['step_name']} → {curr_step['step_name']}: {change_str} features")

# Identificar penhascos
print(f"\n🚨 IDENTIFICAÇÃO DE PENHASCOS:")
penhascos_encontrados = []

for i in range(1, len(pipeline_steps)):
    prev_step = pipeline_steps[i-1]
    curr_step = pipeline_steps[i]
    
    loss = prev_step['shape'][1] - curr_step['shape'][1]
    if loss > 5:  # Perda significativa
        penhascos_encontrados.append({
            'from': prev_step['step_name'],
            'to': curr_step['step_name'],
            'loss': loss
        })

if penhascos_encontrados:
    print(f"   ❌ {len(penhascos_encontrados)} penhasco(s) identificado(s):")
    for penhasco in penhascos_encontrados:
        print(f"      {penhasco['from']} → {penhasco['to']}: -{penhasco['loss']} features")
else:
    print(f"   ✅ Nenhum penhasco significativo identificado")

# Verificação final
print(f"\n✅ VERIFICAÇÃO FINAL:")
print(f"   Dataset inicial: {pipeline_steps[0]['shape']}")
print(f"   Dataset final: {pipeline_steps[-1]['shape']}")
print(f"   Features criadas: {pipeline_steps[-1]['shape'][1] - pipeline_steps[0]['shape'][1]:+d}")
print(f"   Linhas preservadas: {pipeline_steps[-1]['shape'][0] / pipeline_steps[0]['shape'][0]:.1%}")

# Status do pipeline
final_features = pipeline_steps[-1]['shape'][1]
if final_features >= 50:
    status = "✅ EXCELENTE"
elif final_features >= 30:
    status = "✅ BOM"
elif final_features >= 20:
    status = "⚠️ ACEITÁVEL"
else:
    status = "❌ INSUFICIENTE"

print(f"\n🎯 STATUS DO PIPELINE: {status}")
print(f"   Features finais: {final_features}")
print(f"   Pronto para modelagem: {'SIM' if final_features >= 20 else 'NÃO'}")

print(f"\n" + "="*80)
print(f"🚀 PIPELINE EXPLÍCITO CONCLUÍDO COM SUCESSO!")
print(f"="*80)


🔍 AUDITORIA COMPLETA DO PIPELINE EXPLÍCITO

📊 RESUMO COMPLETO DO PIPELINE:
Step                      Shape           Features   Técnicas   Shifted    Candlestick 
-------------------------------------------------------------------------------------
STEP 01 - Dados Brutos    3592x7          7          0          0          0           
STEP 02 - Processamento... 3591x10         10         0          0          0           
STEP 03 - Features Técn... 3591x44         44         17         0          2           
STEP 04 - Seleção de Fe... 3591x40         40         16         0          2           
STEP 05 - Correção Temp... 3590x40         40         18         31         2           
STEP 06 - Padrões de Ca... 3590x49         49         18         31         11          
STEP 07 - Dataset Final   3586x49         49         18         31         11          

📈 EVOLUÇÃO DAS FEATURES:
   STEP 01 - Dados Brutos → STEP 02 - Processamento Básico: +3 features
   STEP 02 - Processamento Bási

## EXPORTAR DATASET FINAL

In [10]:
# EXPORTAR DATASET FINAL
print("\n💾 EXPORTANDO DATASET FINAL...")

# Salvar dataset final
output_filename = 'dataset_final_pipeline_explicito.csv'
df_step07_final.to_csv(output_filename, index=False)

print(f"✅ Dataset exportado: {output_filename}")
print(f"📊 Shape: {df_step07_final.shape}")
print(f"📋 Features: {list(df_step07_final.columns)}")

# Criar relatório de features
report_filename = 'relatorio_features_pipeline.txt'
with open(report_filename, 'w', encoding='utf-8') as f:
    f.write("RELATÓRIO DE FEATURES - PIPELINE EXPLÍCITO\n")
    f.write("="*50 + "\n\n")
    
    f.write(f"Dataset Final: {df_step07_final.shape}\n\n")
    
    f.write("FEATURES POR CATEGORIA:\n")
    f.write("-" * 30 + "\n")
    
    # Categorizar features finais
    final_cols = df_step07_final.columns
    
    ohlc_final = [col for col in final_cols if any(x in col for x in ['Abertura', 'Máxima', 'Mínima', 'Último', 'Volume'])]
    temporal_final = [col for col in final_cols if any(x in col.lower() for x in ['data', 'day', 'month', 'quarter', 'year'])]
    technical_final = [col for col in final_cols if any(x in col.lower() for x in ['ma_', 'bb_', 'rsi', 'atr', 'volatility', 'macd'])]
    shifted_final = [col for col in final_cols if col.endswith('_shifted')]
    candlestick_final = [col for col in final_cols if any(x in col.lower() for x in ['doji', 'hammer', 'engulf', 'prev'])]
    target_final = [col for col in final_cols if 'target' in col.lower()]
    
    f.write(f"OHLC/Volume ({len(ohlc_final)}): {ohlc_final}\n\n")
    f.write(f"Temporais ({len(temporal_final)}): {temporal_final}\n\n")
    f.write(f"Técnicas ({len(technical_final)}): {technical_final}\n\n")
    f.write(f"Shifted ({len(shifted_final)}): {shifted_final}\n\n")
    f.write(f"Candlestick ({len(candlestick_final)}): {candlestick_final}\n\n")
    f.write(f"Target ({len(target_final)}): {target_final}\n\n")

print(f"✅ Relatório criado: {report_filename}")
print(f"\n🎉 PIPELINE EXPLÍCITO CONCLUÍDO COM SUCESSO!")
print(f"📊 {df_step07_final.shape[1]} features preservadas e prontas para modelagem")


💾 EXPORTANDO DATASET FINAL...
✅ Dataset exportado: dataset_final_pipeline_explicito.csv
📊 Shape: (3586, 49)
📋 Features: ['Data', 'day_of_week', 'month', 'quarter', 'year', 'is_month_start', 'is_month_end', 'is_quarter_end', 'Target', 'Abertura_shifted', 'Máxima_shifted', 'Mínima_shifted', 'Último_shifted', 'Volume_shifted', 'MA_5_shifted', 'MA_10_shifted', 'MA_20_shifted', 'MA_50_shifted', 'BB_Upper_shifted', 'BB_Lower_shifted', 'BB_Width_shifted', 'BB_Position_shifted', 'RSI_shifted', 'MACD_shifted', 'Signal_Line_shifted', 'atr_5_shifted', 'atr_10_shifted', 'atr_20_shifted', 'volatility_5_shifted', 'volatility_10_shifted', 'volatility_20_shifted', 'Price_Range_shifted', 'Price_Position_shifted', 'Gap_shifted', 'hl_close_ratio_shifted', 'true_range_shifted', 'high_close_prev_shifted', 'low_close_prev_shifted', 'returns_shifted', 'Variacao_shifted', 'doji_prev', 'hammer_prev', 'shooting_star_prev', 'engulfing_bullish_prev', 'bullish_candle_prev', 'bearish_candle_prev', 'body_size_prev'

# 🏆 OLIMPÍADA DE MODELOS: TESTE COMPLETO DE ALGORITMOS

## Objetivo Duplo:
1. **Encontrar o algoritmo de melhor performance**
2. **Verificar se features de candlestick agregam valor preditivo**

## Estratégia de Teste:
- **Dataset A (Baseline)**: ~40 features (STEP 05 - sem candlestick)
- **Dataset B (Enriquecido)**: ~49 features (STEP 07 - com candlestick)

## Categorias de Modelos:
1. **Baseline Linear**: Regressão Logística
2. **Ensembles de Árvores**: Random Forest, XGBoost, LightGBM
3. **Não-Lineares**: SVM (RBF)
4. **Ensemble Híbrido**: Voting Classifier

In [1]:
# PREPARAÇÃO PARA OLIMPÍADA DE MODELOS
print("🏆 === OLIMPÍADA DE MODELOS === 🏆")
print()
print("Preparando framework de testes completo...")

# Imports necessários
from sklearn.model_selection import TimeSeriesSplit, cross_val_score, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, roc_auc_score
import xgboost as xgb
import lightgbm as lgb
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import time

# Função para avaliação completa de modelos
def evaluate_model_complete(model, X_train, X_test, y_train, y_test, model_name, dataset_name):
    """
    Avaliação completa de um modelo com múltiplas métricas
    """
    start_time = time.time()
    
    # Treinar modelo
    model.fit(X_train, y_train)
    
    # Predições
    y_pred = model.predict(X_test)
    y_pred_proba = model.predict_proba(X_test)[:, 1] if hasattr(model, 'predict_proba') else None
    
    # Métricas
    accuracy = accuracy_score(y_test, y_pred)
    auc_score = roc_auc_score(y_test, y_pred_proba) if y_pred_proba is not None else None
    
    # Tempo de treinamento
    training_time = time.time() - start_time
    
    # Cross-validation com TimeSeriesSplit
    tscv = TimeSeriesSplit(n_splits=5)
    cv_scores = cross_val_score(model, X_train, y_train, cv=tscv, scoring='accuracy')
    
    return {
        'model_name': model_name,
        'dataset': dataset_name,
        'accuracy': accuracy,
        'auc_score': auc_score,
        'cv_mean': cv_scores.mean(),
        'cv_std': cv_scores.std(),
        'training_time': training_time,
        'y_pred': y_pred,
        'y_pred_proba': y_pred_proba,
        'model': model
    }

# Função para preparar datasets
def prepare_datasets():
    """
    Prepara Dataset A (baseline) e Dataset B (enriquecido)
    """
    print("📊 Preparando datasets para competição...")
    
    # Dataset A (Baseline) - STEP 05 (sem candlestick)
    dataset_a = df_step05_temporal.copy()
    
    # Dataset B (Enriquecido) - STEP 07 (com candlestick)
    dataset_b = df_step07_final.copy()
    
    print(f"   Dataset A (Baseline): {dataset_a.shape}")
    print(f"   Dataset B (Enriquecido): {dataset_b.shape}")
    print(f"   Diferença: +{dataset_b.shape[1] - dataset_a.shape[1]} features no Dataset B")
    
    return dataset_a, dataset_b

# Função para split temporal
def create_temporal_split(df, test_size=0.2):
    """
    Cria split temporal respeitando ordem cronológica
    """
    # Ordenar por data se disponível
    if 'Data' in df.columns:
        df_sorted = df.sort_values('Data').reset_index(drop=True)
    else:
        df_sorted = df.copy()
    
    # Split temporal
    split_idx = int(len(df_sorted) * (1 - test_size))
    
    train_df = df_sorted.iloc[:split_idx]
    test_df = df_sorted.iloc[split_idx:]
    
    # Separar features e target
    feature_cols = [col for col in df_sorted.columns if col not in ['Data', 'Target']]
    
    X_train = train_df[feature_cols]
    X_test = test_df[feature_cols]
    y_train = train_df['Target']
    y_test = test_df['Target']
    
    return X_train, X_test, y_train, y_test

print("✅ Framework de avaliação preparado")
print("🏁 Pronto para iniciar a competição!")

🏆 === OLIMPÍADA DE MODELOS === 🏆

Preparando framework de testes completo...
✅ Framework de avaliação preparado
🏁 Pronto para iniciar a competição!


## PREPARAÇÃO DOS DATASETS E COMPETIDORES

In [2]:
# PREPARAÇÃO DOS DATASETS
print("📊 === PREPARAÇÃO DOS DATASETS === 📊")

# Preparar datasets A e B
dataset_a, dataset_b = prepare_datasets()

# Criar splits temporais para ambos datasets
print(f"\n🔄 Criando splits temporais...")

X_train_a, X_test_a, y_train_a, y_test_a = create_temporal_split(dataset_a)
X_train_b, X_test_b, y_train_b, y_test_b = create_temporal_split(dataset_b)

print(f"\n📋 RESUMO DOS SPLITS:")
print(f"Dataset A - Train: {X_train_a.shape}, Test: {X_test_a.shape}")
print(f"Dataset B - Train: {X_train_b.shape}, Test: {X_test_b.shape}")

# Verificar distribuição do target
print(f"\n🎯 DISTRIBUIÇÃO DO TARGET:")
print(f"Dataset A - Train: {y_train_a.value_counts(normalize=True).to_dict()}")
print(f"Dataset A - Test: {y_test_a.value_counts(normalize=True).to_dict()}")
print(f"Dataset B - Train: {y_train_b.value_counts(normalize=True).to_dict()}")
print(f"Dataset B - Test: {y_test_b.value_counts(normalize=True).to_dict()}")

# Preparar competidores
print(f"\n🏆 === PREPARAÇÃO DOS COMPETIDORES === 🏆")

# Categoria 1: Baseline Linear
competitors = {
    'Regressão Logística': Pipeline([
        ('scaler', StandardScaler()),
        ('logreg', LogisticRegression(C=0.1, solver='liblinear', random_state=42))
    ]),
    
    # Categoria 2: Ensembles de Árvores
    'Random Forest': RandomForestClassifier(
        n_estimators=100,
        max_depth=10,
        min_samples_split=5,
        min_samples_leaf=2,
        random_state=42,
        n_jobs=-1
    ),
    
    'XGBoost': xgb.XGBClassifier(
        n_estimators=100,
        max_depth=6,
        learning_rate=0.1,
        subsample=0.8,
        colsample_bytree=0.8,
        random_state=42,
        eval_metric='logloss'
    ),
    
    'LightGBM': lgb.LGBMClassifier(
        n_estimators=100,
        max_depth=6,
        learning_rate=0.1,
        subsample=0.8,
        colsample_bytree=0.8,
        random_state=42,
        verbose=-1
    ),
    
    # Categoria 3: Não-Lineares
    'SVM (RBF)': Pipeline([
        ('scaler', StandardScaler()),
        ('svm', SVC(kernel='rbf', probability=True, random_state=42, C=1.0))
    ])
}

print(f"✅ {len(competitors)} competidores preparados:")
for name in competitors.keys():
    print(f"   🤖 {name}")

print(f"\n🚀 Tudo pronto para a Olimpíada de Modelos!")

📊 === PREPARAÇÃO DOS DATASETS === 📊
📊 Preparando datasets para competição...


NameError: name 'df_step05_temporal' is not defined

## EXECUÇÃO DA OLIMPÍADA DE MODELOS

In [ ]:
# EXECUÇÃO DA OLIMPÍADA DE MODELOS
print("🏆 === INICIANDO OLIMPÍADA DE MODELOS === 🏆")
print(f"Timestamp: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print()

# Armazenar todos os resultados
all_results = []

# Executar cada modelo em ambos datasets
for model_name, model in competitors.items():
    print(f"\n🤖 === TESTANDO: {model_name.upper()} === 🤖")
    
    # Teste no Dataset A (Baseline)
    print(f"\n📊 Dataset A (Baseline - {X_train_a.shape[1]} features):")
    try:
        result_a = evaluate_model_complete(
            model, X_train_a, X_test_a, y_train_a, y_test_a, 
            model_name, "Dataset A (Baseline)"
        )
        all_results.append(result_a)
        
        print(f"   ✅ Accuracy: {result_a['accuracy']:.4f}")
        print(f"   📈 AUC: {result_a['auc_score']:.4f}" if result_a['auc_score'] else "   📈 AUC: N/A")
        print(f"   🔄 CV Mean: {result_a['cv_mean']:.4f} (±{result_a['cv_std']:.4f})")
        print(f"   ⏱️ Tempo: {result_a['training_time']:.2f}s")
        
    except Exception as e:
        print(f"   ❌ Erro no Dataset A: {e}")
        continue
    
    # Teste no Dataset B (Enriquecido)
    print(f"\n📊 Dataset B (Enriquecido - {X_train_b.shape[1]} features):")
    try:
        result_b = evaluate_model_complete(
            model, X_train_b, X_test_b, y_train_b, y_test_b, 
            model_name, "Dataset B (Enriquecido)"
        )
        all_results.append(result_b)
        
        print(f"   ✅ Accuracy: {result_b['accuracy']:.4f}")
        print(f"   📈 AUC: {result_b['auc_score']:.4f}" if result_b['auc_score'] else "   📈 AUC: N/A")
        print(f"   🔄 CV Mean: {result_b['cv_mean']:.4f} (±{result_b['cv_std']:.4f})")
        print(f"   ⏱️ Tempo: {result_b['training_time']:.2f}s")
        
        # Comparar com Dataset A
        accuracy_diff = result_b['accuracy'] - result_a['accuracy']
        auc_diff = (result_b['auc_score'] - result_a['auc_score']) if (result_b['auc_score'] and result_a['auc_score']) else None
        
        print(f"\n📊 COMPARAÇÃO (B vs A):")
        print(f"   📈 Accuracy: {accuracy_diff:+.4f} ({'✅ Melhora' if accuracy_diff > 0 else '❌ Piora' if accuracy_diff < 0 else '➖ Igual'})")
        if auc_diff is not None:
            print(f"   📈 AUC: {auc_diff:+.4f} ({'✅ Melhora' if auc_diff > 0 else '❌ Piora' if auc_diff < 0 else '➖ Igual'})")
        
    except Exception as e:
        print(f"   ❌ Erro no Dataset B: {e}")
        continue
    
    print(f"\n{'='*60}")

print(f"\n🏁 OLIMPÍADA CONCLUÍDA!")
print(f"📊 Total de resultados coletados: {len(all_results)}")

## ANÁLISE COMPLETA DOS RESULTADOS

In [ ]:
# ANÁLISE COMPLETA DOS RESULTADOS
print("📊 === ANÁLISE COMPLETA DOS RESULTADOS === 📊")
print()

if len(all_results) > 0:
    # Criar DataFrame com resultados
    results_df = pd.DataFrame(all_results)
    
    # Tabela de resultados
    print("🏆 TABELA DE RESULTADOS COMPLETA:")
    print("="*100)
    print(f"{'Modelo':<20} {'Dataset':<25} {'Accuracy':<10} {'AUC':<8} {'CV Mean':<10} {'CV Std':<8} {'Tempo(s)':<8}")
    print("-"*100)
    
    for _, row in results_df.iterrows():
        auc_str = f"{row['auc_score']:.4f}" if row['auc_score'] is not None else "N/A"
        print(f"{row['model_name']:<20} {row['dataset']:<25} {row['accuracy']:<10.4f} {auc_str:<8} {row['cv_mean']:<10.4f} {row['cv_std']:<8.4f} {row['training_time']:<8.2f}")
    
    print()
    
    # Análise por dataset
    print("📊 RANKING POR DATASET:")
    print()
    
    for dataset in ['Dataset A (Baseline)', 'Dataset B (Enriquecido)']:
        dataset_results = results_df[results_df['dataset'] == dataset].copy()
        if len(dataset_results) > 0:
            dataset_results = dataset_results.sort_values('accuracy', ascending=False)
            
            print(f"🏅 {dataset}:")
            for i, (_, row) in enumerate(dataset_results.iterrows(), 1):
                medal = "🥇" if i == 1 else "🥈" if i == 2 else "🥉" if i == 3 else f"{i}º"
                auc_str = f"AUC: {row['auc_score']:.4f}" if row['auc_score'] is not None else "AUC: N/A"
                print(f"   {medal} {row['model_name']}: Acc: {row['accuracy']:.4f}, {auc_str}")
            print()
    
    # Análise do impacto das features de candlestick
    print("🕯️ ANÁLISE DO IMPACTO DAS FEATURES DE CANDLESTICK:")
    print("="*60)
    
    candlestick_impact = []
    
    for model_name in results_df['model_name'].unique():
        model_results = results_df[results_df['model_name'] == model_name]
        
        if len(model_results) == 2:  # Tem resultado para ambos datasets
            baseline = model_results[model_results['dataset'] == 'Dataset A (Baseline)'].iloc[0]
            enriched = model_results[model_results['dataset'] == 'Dataset B (Enriquecido)'].iloc[0]
            
            accuracy_improvement = enriched['accuracy'] - baseline['accuracy']
            auc_improvement = (enriched['auc_score'] - baseline['auc_score']) if (enriched['auc_score'] and baseline['auc_score']) else None
            
            candlestick_impact.append({
                'model': model_name,
                'accuracy_improvement': accuracy_improvement,
                'auc_improvement': auc_improvement,
                'baseline_accuracy': baseline['accuracy'],
                'enriched_accuracy': enriched['accuracy']
            })
            
            impact_symbol = "✅" if accuracy_improvement > 0.01 else "⚠️" if accuracy_improvement > 0 else "❌"
            print(f"{impact_symbol} {model_name}:")
            print(f"   Accuracy: {baseline['accuracy']:.4f} → {enriched['accuracy']:.4f} ({accuracy_improvement:+.4f})")
            if auc_improvement is not None:
                print(f"   AUC: {baseline['auc_score']:.4f} → {enriched['auc_score']:.4f} ({auc_improvement:+.4f})")
            print()
    
    # Resumo do impacto das features de candlestick
    if candlestick_impact:
        avg_accuracy_improvement = np.mean([x['accuracy_improvement'] for x in candlestick_impact])
        positive_improvements = sum(1 for x in candlestick_impact if x['accuracy_improvement'] > 0)
        
        print(f"📈 RESUMO DO IMPACTO DAS FEATURES DE CANDLESTICK:")
        print(f"   Melhoria média de accuracy: {avg_accuracy_improvement:+.4f}")
        print(f"   Modelos que melhoraram: {positive_improvements}/{len(candlestick_impact)}")
        
        if avg_accuracy_improvement > 0.01:
            conclusion = "✅ FEATURES DE CANDLESTICK AGREGAM VALOR SIGNIFICATIVO"
        elif avg_accuracy_improvement > 0:
            conclusion = "⚠️ FEATURES DE CANDLESTICK AGREGAM VALOR MARGINAL"
        else:
            conclusion = "❌ FEATURES DE CANDLESTICK NÃO AGREGAM VALOR"
        
        print(f"\n🎯 CONCLUSÃO: {conclusion}")
    
    # Identificar o campeão geral
    print(f"\n🏆 CAMPEÃO GERAL:")
    best_result = results_df.loc[results_df['accuracy'].idxmax()]
    print(f"   🥇 {best_result['model_name']} no {best_result['dataset']}")
    print(f"   📊 Accuracy: {best_result['accuracy']:.4f}")
    print(f"   📈 AUC: {best_result['auc_score']:.4f}" if best_result['auc_score'] else "   📈 AUC: N/A")
    print(f"   🔄 CV: {best_result['cv_mean']:.4f} (±{best_result['cv_std']:.4f})")
    
else:
    print("❌ Nenhum resultado coletado. Verifique a execução dos modelos.")

## ENSEMBLE HÍBRIDO: COMBINANDO OS MELHORES

In [ ]:
# ENSEMBLE HÍBRIDO: COMBINANDO OS MELHORES
print("🤝 === ENSEMBLE HÍBRIDO: COMBINANDO OS MELHORES === 🤝")
print()

if len(all_results) >= 3:
    # Identificar os 3 melhores modelos no Dataset B
    dataset_b_results = [r for r in all_results if r['dataset'] == 'Dataset B (Enriquecido)']
    
    if len(dataset_b_results) >= 3:
        # Ordenar por accuracy
        dataset_b_results.sort(key=lambda x: x['accuracy'], reverse=True)
        top_3 = dataset_b_results[:3]
        
        print(f"🏅 TOP 3 MODELOS SELECIONADOS PARA ENSEMBLE:")
        for i, result in enumerate(top_3, 1):
            print(f"   {i}º {result['model_name']}: {result['accuracy']:.4f}")
        
        # Criar Voting Classifier com os top 3
        ensemble_estimators = []
        
        for result in top_3:
            model_name = result['model_name']
            model = result['model']
            
            # Criar nome único para o ensemble
            ensemble_name = model_name.lower().replace(' ', '_').replace('(', '').replace(')', '')
            ensemble_estimators.append((ensemble_name, model))
        
        # Criar Voting Classifier
        voting_classifier = VotingClassifier(
            estimators=ensemble_estimators,
            voting='soft'  # Usa probabilidades
        )
        
        print(f"\n🤖 Testando Voting Classifier (Soft Voting)...")
        
        # Testar ensemble no Dataset B
        try:
            ensemble_result = evaluate_model_complete(
                voting_classifier, X_train_b, X_test_b, y_train_b, y_test_b,
                "Voting Classifier", "Dataset B (Enriquecido)"
            )
            
            print(f"\n📊 RESULTADO DO ENSEMBLE:")
            print(f"   ✅ Accuracy: {ensemble_result['accuracy']:.4f}")
            print(f"   📈 AUC: {ensemble_result['auc_score']:.4f}")
            print(f"   🔄 CV Mean: {ensemble_result['cv_mean']:.4f} (±{ensemble_result['cv_std']:.4f})")
            print(f"   ⏱️ Tempo: {ensemble_result['training_time']:.2f}s")
            
            # Comparar com o melhor individual
            best_individual = top_3[0]
            improvement = ensemble_result['accuracy'] - best_individual['accuracy']
            
            print(f"\n🆚 ENSEMBLE vs MELHOR INDIVIDUAL:")
            print(f"   Melhor individual: {best_individual['model_name']} ({best_individual['accuracy']:.4f})")
            print(f"   Ensemble: {ensemble_result['accuracy']:.4f}")
            print(f"   Diferença: {improvement:+.4f} ({'✅ Melhora' if improvement > 0 else '❌ Piora' if improvement < 0 else '➖ Igual'})")
            
            # Adicionar resultado do ensemble à lista
            all_results.append(ensemble_result)
            
        except Exception as e:
            print(f"❌ Erro ao criar ensemble: {e}")
    
    else:
        print(f"⚠️ Poucos resultados no Dataset B para criar ensemble")
else:
    print(f"⚠️ Poucos resultados coletados para criar ensemble")

print(f"\n{'='*60}")

## VISUALIZAÇÕES E RELATÓRIO FINAL

In [ ]:
# VISUALIZAÇÕES E RELATÓRIO FINAL
print("📊 === VISUALIZAÇÕES E RELATÓRIO FINAL === 📊")
print()

if len(all_results) > 0:
    # Criar DataFrame atualizado com ensemble
    final_results_df = pd.DataFrame(all_results)
    
    # Gráfico de comparação de accuracy
    plt.figure(figsize=(15, 8))
    
    # Subplot 1: Comparação de Accuracy por Modelo e Dataset
    plt.subplot(2, 2, 1)
    
    # Preparar dados para o gráfico
    models = final_results_df['model_name'].unique()
    datasets = final_results_df['dataset'].unique()
    
    x = np.arange(len(models))
    width = 0.35
    
    for i, dataset in enumerate(datasets):
        dataset_data = final_results_df[final_results_df['dataset'] == dataset]
        accuracies = []
        
        for model in models:
            model_data = dataset_data[dataset_data['model_name'] == model]
            if len(model_data) > 0:
                accuracies.append(model_data.iloc[0]['accuracy'])
            else:
                accuracies.append(0)
        
        plt.bar(x + i*width, accuracies, width, label=dataset, alpha=0.8)
    
    plt.xlabel('Modelos')
    plt.ylabel('Accuracy')
    plt.title('Comparação de Accuracy por Modelo e Dataset')
    plt.xticks(x + width/2, models, rotation=45, ha='right')
    plt.legend()
    plt.grid(True, alpha=0.3)
    
    # Subplot 2: Impacto das Features de Candlestick
    plt.subplot(2, 2, 2)
    
    if len(candlestick_impact) > 0:
        models_impact = [x['model'] for x in candlestick_impact]
        improvements = [x['accuracy_improvement'] for x in candlestick_impact]
        
        colors = ['green' if x > 0 else 'red' for x in improvements]
        plt.bar(models_impact, improvements, color=colors, alpha=0.7)
        plt.xlabel('Modelos')
        plt.ylabel('Melhoria de Accuracy')
        plt.title('Impacto das Features de Candlestick')
        plt.xticks(rotation=45, ha='right')
        plt.axhline(y=0, color='black', linestyle='-', alpha=0.3)
        plt.grid(True, alpha=0.3)
    
    # Subplot 3: Distribuição de AUC
    plt.subplot(2, 2, 3)
    
    auc_scores = [r['auc_score'] for r in all_results if r['auc_score'] is not None]
    if auc_scores:
        plt.hist(auc_scores, bins=10, alpha=0.7, color='skyblue', edgecolor='black')
        plt.xlabel('AUC Score')
        plt.ylabel('Frequência')
        plt.title('Distribuição de AUC Scores')
        plt.grid(True, alpha=0.3)
    
    # Subplot 4: Tempo de Treinamento
    plt.subplot(2, 2, 4)
    
    model_names = [r['model_name'] for r in all_results]
    training_times = [r['training_time'] for r in all_results]
    
    plt.scatter(training_times, [r['accuracy'] for r in all_results], alpha=0.7, s=100)
    
    for i, model in enumerate(model_names):
        plt.annotate(model[:10], (training_times[i], all_results[i]['accuracy']), 
                    xytext=(5, 5), textcoords='offset points', fontsize=8)
    
    plt.xlabel('Tempo de Treinamento (s)')
    plt.ylabel('Accuracy')
    plt.title('Accuracy vs Tempo de Treinamento')
    plt.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # Relatório final em texto
    print(f"\n📋 === RELATÓRIO FINAL DA OLIMPÍADA === 📋")
    print(f"Timestamp: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    print(f"="*60)
    
    # Campeão absoluto
    best_overall = final_results_df.loc[final_results_df['accuracy'].idxmax()]
    print(f"\n🏆 CAMPEÃO ABSOLUTO:")
    print(f"   Modelo: {best_overall['model_name']}")
    print(f"   Dataset: {best_overall['dataset']}")
    print(f"   Accuracy: {best_overall['accuracy']:.4f}")
    print(f"   AUC: {best_overall['auc_score']:.4f}" if best_overall['auc_score'] else "   AUC: N/A")
    
    # Melhor por categoria
    print(f"\n🏅 MELHORES POR CATEGORIA:")
    
    categories = {
        'Linear': ['Regressão Logística'],
        'Árvores': ['Random Forest', 'XGBoost', 'LightGBM'],
        'Não-Linear': ['SVM (RBF)'],
        'Ensemble': ['Voting Classifier']
    }
    
    for category, models in categories.items():
        category_results = final_results_df[final_results_df['model_name'].isin(models)]
        if len(category_results) > 0:
            best_in_category = category_results.loc[category_results['accuracy'].idxmax()]
            print(f"   {category}: {best_in_category['model_name']} ({best_in_category['accuracy']:.4f})")
    
    # Conclusão sobre features de candlestick
    if candlestick_impact:
        avg_improvement = np.mean([x['accuracy_improvement'] for x in candlestick_impact])
        print(f"\n🕯️ CONCLUSÃO SOBRE FEATURES DE CANDLESTICK:")
        print(f"   Melhoria média: {avg_improvement:+.4f}")
        
        if avg_improvement > 0.01:
            print(f"   ✅ RECOMENDAÇÃO: Usar features de candlestick (melhoria significativa)")
        elif avg_improvement > 0:
            print(f"   ⚠️ RECOMENDAÇÃO: Considerar features de candlestick (melhoria marginal)")
        else:
            print(f"   ❌ RECOMENDAÇÃO: Não usar features de candlestick (sem benefício)")
    
    # Recomendação final
    print(f"\n🎯 RECOMENDAÇÃO FINAL PARA PRODUÇÃO:")
    print(f"   Modelo: {best_overall['model_name']}")
    print(f"   Dataset: {best_overall['dataset']}")
    print(f"   Justificativa: Melhor accuracy ({best_overall['accuracy']:.4f}) na competição")
    
    print(f"\n🎉 OLIMPÍADA DE MODELOS CONCLUÍDA COM SUCESSO!")
    print(f"="*60)

else:
    print(f"❌ Nenhum resultado disponível para análise")